# Recommendation Systems using Steam Data - Data Extraction 2 <a id='top'></a>

In this notebook, we perform our second data extraction. This time, we create a dataset with a focus on the game reviews. To do so, we started by extracting data using the Steam API. In this notebook, data related with the each games is going to be extracted. The final columns of our dataset are:
* aa
* aa

The structure of this notebook is as follows:

[0. Import Libraries](#libraries) <br>
[1. Import Topsellers File](#topseller) <br>
[2. Extract Data](#extract) <br>
&emsp; [2.1. Check Current Data](#current_data) <br>
&emsp; [2.2. Extract Data](#extract_data) <br>


# 0. Import Libraries<a id='libraries'></a>
[to the top](#top)  

The first step is to import the necessary libraries.

In [35]:
import polars as pl
import os
import time
from urllib.parse import quote

In [36]:
def get_steam_reviews(app_id, cursor='*'):
    url = f"https://store.steampowered.com/appreviews/{app_id}?json=1&filter=recent&language=english&cursor={cursor}&num_per_page=100"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        if data['success'] == 1:
            return data
        else:
            print(f"Failed to get valid reviews for app ID {app_id}.")
            print(data)
            return data
    else:
        print(f"Failed to get reviews for app ID {app_id}. Status code: {response.status_code}")
        return None

In [37]:
def fetch_all_reviews(app_id):

    cursor = '*'
    all_reviews = []
    
    while True:
        data = get_steam_reviews(app_id, cursor)
        if data is None or data['query_summary']['num_reviews'] == 0:
            break
        
        # Collect reviews
        reviews = data['reviews']
        all_reviews.extend(reviews)
        
        # Update cursor
        cursor = quote(data['cursor'])
    
    # Create a DataFrame from reviews
    reviews_df = pl.DataFrame(all_reviews)
    num_entries = len(reviews_df)

    # Specify the directory and file name for the parquet file
    directory = "data\\parquets"
    file_name = f"{app_id}_reviews_{num_entries}.parquet"
    file_path = os.path.join(directory, file_name)
    
    # Save to parquet
    reviews_df.write_parquet(file_path)

    return num_entries

# Single fetch for specific game

In [ ]:
# start_time = time.time()
# num_entries = fetch_all_reviews(553850)
# elapsed_time = time.time() - start_time

# print(f"Die Ausführungszeit betrug {elapsed_time:.2f} Sekunden.")
# print(f"Anzahl Einträge:{num_entries}")

# 1. Import Topsellers File<a id='topseller'></a>
[to the top](#top)  

As we wanted to get data from the topseller games, we import a json file that has the APP IDs of the top seller games.

In [ ]:
topseller = [x for x in list(pl.read_json('data/jsons/SteamTopSellers.json'))[0]]
topseller[:5] 

In [39]:
mostplayed = [x for x in list(pl.read_json('data/jsons/SteamMostPlayed.json'))[1]]
mostplayed[:5] 

['730', '570', '1085660', '578080', '1172470']

In [ ]:
# monthlytopgames = [x for x in list(pl.read_json('data/jsons/MonthlyTopGames.json'))[0]]
# monthlytopgames[:5] 

# 2. Extract Data<a id='extract'></a>
[to the top](#top)  

After selecting which games we want data on, it is time to start extracting the data.

## 2.1. Check Current Data<a id='current_data'></a>
[to the top](#top)  

As extracting data is a long process, we decided to created a way to check which games we already have data on and extract data on the ones that we do not have yet. To do so, we create a list of the APP IDs of the games that we already extracted data on.

In [40]:
folder_path = 'data\\parquets'
downloaded = []
for filename in os.listdir(folder_path):
    if filename.endswith('.parquet'):
        game = filename.split('_')[0]
        downloaded.append(game)
len(downloaded)

596

## 2.2. Extract Data<a id='extract_data'></a>
[to the top](#top)  

Below, we check if we already have data on each game. If we do not, we proceed to extract it. It also checks how long does it take to extract the last batch of data.

In [ ]:
# for game in topseller:
#     if game in downloaded:
#         continue
#     start_time = time.time()
#     num_entries = fetch_all_reviews(game)
#     elapsed_time = time.time() - start_time

#     # Conversion of time from seconds to hours, minutes, and seconds
#     hours, remainder = divmod(elapsed_time, 3600)
#     minutes, seconds = divmod(remainder, 60)

#     print(f"""AppID: {game}
#     Duration: {int(hours)} hours, {int(minutes)} minutes, {seconds:.2f} seconds
#     Count: {num_entries}
#     ####################""")

In [41]:
for game in mostplayed:
    if game in downloaded:
        continue
    start_time = time.time()
    num_entries = fetch_all_reviews(game)
    elapsed_time = time.time() - start_time

    # Conversion of time from seconds to hours, minutes, and seconds
    hours, remainder = divmod(elapsed_time, 3600)
    minutes, seconds = divmod(remainder, 60)

    print(f"""AppID: {game}
    Duration: {int(hours)} hours, {int(minutes)} minutes, {seconds:.2f} seconds
    Count: {num_entries}
    ####################""")

In [ ]:
# for game in monthlytopgames:
#     if game in downloaded:
#         continue
#     start_time = time.time()
#     num_entries = fetch_all_reviews(game)
#     elapsed_time = time.time() - start_time

#     # Conversion of time from seconds to hours, minutes, and seconds
#     hours, remainder = divmod(elapsed_time, 3600)
#     minutes, seconds = divmod(remainder, 60)

#     print(f"""AppID: {game}
#     Duration: {int(hours)} hours, {int(minutes)} minutes, {seconds:.2f} seconds
#     Count: {num_entries}
#     ####################""")